In [29]:
import pandas as pd
# import pickle
import sqlalchemy
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.downloader.download('vader_lexicon')
nltk.downloader.download('punkt')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Gavin\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Gavin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [30]:
# Import the 10-K Data from the Database
db_name = "SEC_Filings.db"
table_name = "10K_Data"

engine = sqlalchemy.create_engine('sqlite:///' + db_name, execution_options={"sqlite_raw_colnames": True})
df = pd.read_sql_table(table_name, engine)

# Clean DataFrame
agri = ['0100', '0200', '0900']
fin = ['6172', '6199', '6200', '6211', '6221', '6282']
air = ['4513', '4522', '4581']
bank = ['6022', '6029', '6035', '6036', '6099']

df['Industry'] = ''
df.loc[df['SIC'].isin(agri), 'Industry'] = "Agriculture"
df.loc[df['SIC'].isin(fin), 'Industry'] = "Financial Services"
df.loc[df['SIC'].isin(air), 'Industry'] = "Aviation"
df.loc[df['SIC'].isin(bank), 'Industry'] = "Banking"

df = df.drop(['Filing_Link', 'Accession_No', 'SIC', 'Company'], axis=1)

In [50]:
# Sentiment Analysis
# Set 'Aspect' Words
word_search_list_2 = [{'artificial intelligence': ['artificial intelligence', 'artificial-intelligence', ' ai.', ' ai,', ' ai ']},
                      {'deep learning': ['deep learning', 'deep-learning']}, 
                      {'computer vision': ['computer vision', 'computer-vision']}, 
                      'bots', 
                      'automation', 
                      {'machine learning': ['machine learning', 'machine-learning']},
                      'algorithm']
# aspects = [list(x.values())[0] if isinstance (x, dict) else x for x in word_search_list_2]

# Convert 10-K Text to String
# review_text = df['Filing_Document_Text'][0]

# State Sentiment Analyzer - Rule-based Approach
sia = SentimentIntensityAnalyzer()

# Create DataFrame to Store Output
df_sentiments = pd.DataFrame(columns=['Industry', 'Year', 'Aspect', 'Count', 'Sentiment'])

df['Filing_Date'] = pd.to_datetime(df['Filing_Date'], format='%Y-%m-%d')
# Perform Sentiment Analysis

out_list = []
for i, review_text in enumerate(df['Filing_Document_Text']):

    # Create Dictionary to Store Sentiment Scores
    aspect_sentiments = {}
    for aspect_item in word_search_list_2:
        if isinstance(aspect_item, dict):
            aspect = list(aspect_item.keys())[0]
            aspect_sentiments[aspect] = {'count': 0, 'sentiment': 0}
        else:
            aspect_sentiments[aspect_item] = {'count': 0, 'sentiment': 0}

    year = df.loc[i,'Filing_Date'].year
    industry = df.loc[i,'Industry']
    
    sentences = sent_tokenize(review_text)
    for sentence in sentences:
        for aspect_item in word_search_list_2:
            if isinstance (aspect_item, dict):
                aspect = list(aspect_item.keys())[0]
                keywords = list(aspect_item.values())[0]
                for keyword in keywords:
                    if keyword in sentence.lower():
                        sent_score = sia.polarity_scores(sentence)
                        aspect_sentiments[aspect]['count'] += 1
                        aspect_sentiments[aspect]['sentiment'] += sent_score['compound']
            else:
                aspect = aspect_item
                if aspect in sentence.lower():
                    sent_score = sia.polarity_scores(sentence)
                    aspect_sentiments[aspect]['count'] += 1
                    aspect_sentiments[aspect]['sentiment'] += sent_score['compound']
    
    for aspect in aspect_sentiments:
        count = aspect_sentiments[aspect]['count']
        sent = aspect_sentiments[aspect]['sentiment']
        out_data = {'Industry': industry, 'Year': year, 'Aspect': aspect, 'Count': count, 'Sentiment': sent}
        out_list.append(out_data)

df_sentiments = pd.DataFrame.from_dict(out_list)

In [109]:
pd.set_option('display.max_rows', None)

df_agg = df_sentiments[['Industry', 'Aspect', 'Year', 'Count', 'Sentiment']].groupby(['Industry', 'Aspect', 'Year']).sum()
df_agg['Aggregated Sentiment'] = 0
df_agg.loc[df_agg['Count']!=0, 'Aggregated Sentiment'] = df_agg['Sentiment']/df_agg['Count']
df_agg.reset_index(drop=False, inplace=True)
df_agg.drop('Sentiment', axis=1, inplace=True)

df_agg = df_agg.loc[~df_agg['Aspect'].isin(['computer vision', 'deep learning'])].reset_index(drop=True)
df_agg=df_agg.loc[df_agg['Count'] > 5].reset_index(drop=True)

industry_counts = df_sentiments[['Industry', 'Year']].groupby('Industry').count()/len(word_search_list_2)
for industry in df_agg['Industry'].unique():
    df_agg.loc[df_agg['Industry']==industry, 'Average Mentions'] = df_agg['Count']/industry_counts.loc[industry].values

df_agg.to_sql("Aggregated_Sentiment_Scores", engine, if_exists='replace', index=False)